In [20]:
import os
import pandas as pd
from dotenv import load_dotenv
from utils.coin import CoinGeckoAPI  
from utils.manager import BitcoinDatabase  

load_dotenv()

db_name = os.getenv("BTC_DB")
table_btc_price_history = os.getenv("BTC_PRICE_TABLE")
table_rolling_price_history = os.getenv("ROLLING_PRICE_TABLE")



##  Load the data from the db 

In [26]:
btc_db = BitcoinDatabase(db_name,table_btc_price_history)
btc_history_data = btc_db.select_data(table_btc_price_history)



## Find peak values calculating std

If a value is greater or less than two times the standard deviation, it can be considered an anomaly 

In [30]:
df = btc_history_data.copy()

In [32]:
df.head()

,date,price
0,01-01-2022 20:00:00,47816.0777
1,02-01-2022 20:00:00,47387.2122
2,03-01-2022 20:00:00,46531.1409
3,04-01-2022 20:00:00,45938.0243
4,05-01-2022 20:00:00,43647.1475


In [33]:
desviacion_estandar = df['price'].std()

# Encontrar los valores que están por encima o por debajo de 2 desviaciones estándar
valores_anomalos = df[(df['price'] > (df['price'].mean() + 2 * desviacion_estandar)) | 
                      (df['price'] < (df['price'].mean() - 2 * desviacion_estandar))]

# Etiquetar los valores anómalos en el DataFrame original
df['anomalo'] = False
df.loc[valores_anomalos.index, 'anomalo'] = True




## Anomaly values

In [35]:
df[df['anomalo']==True]

,date,price,anomalo
0,01-01-2022 20:00:00,47816.0777,True
1,02-01-2022 20:00:00,47387.2122,True
21,22-01-2022 20:00:00,35180.4355,True
87,29-03-2022 20:00:00,47459.2612,True


## Calculating the IQR 

In [36]:
# Calcular el primer y tercer cuartil
q1 = df['price'].quantile(0.25)
q3 = df['price'].quantile(0.75)

# Calcular el rango intercuartil
iqr = q3 - q1

# Imprimir el rango intercuartil
print("El rango intercuartil es:", iqr)

El rango intercuartil es: 4045.9198749999996


## According the IQR obtained the 50% of the values  have a spread of  4045.919$